## Reed-Solomon/Reed-Muller Product Code

In [292]:
class RSRMCode:
    
    def __init__(self, n_RS, k_RS, q_RS, r_RM, m_RM, alpha = None):
        
        if not (k_RS < n_RS and n_RS <= q_RS):
            raise ValueError('Invalid RS values for n, k, and q.')
            
        self.p0, self.k_RM = is_prime_power(q_RS, get_data = True)
            
        self.n_RS = n_RS
        self.k_RS = k_RS
        self.q_RS = q_RS
        
        self.r_RM = r_RM
        self.m_RM = m_RM
        
        self.n_RM = 2**self.m_RM
        self.k_RM = sum(binomial(self.m_RM,i) for i in range(self.r_RM + 1))
        self.d_RM = 2**(self.m_RM - self.r_RM)
        
        self.d_RS = self.n_RS - self.k_RS + 1
        
        self.tau_RS = floor((self.n_RS-self.k_RS)/2)
        
        # Initializing field for RS
        self.EF = GF(self.q_RS) # extension field
        self.R = PolynomialRing(self.EF, 'X')
        self.p = self.EF.primitive_element()
        
        # Constructing alpha-vector
        if not alpha:
            self.alpha = vector([self.p**i for i in range(self.n_RS)])
        else:
            self.alpha = alpha
        
        # Constructing RS generator matrix
        self.G_RS = matrix(self.EF, self.k_RS, self.n_RS, lambda i,j : self.alpha[j]**i)
        
        # Initializing field for RM
        self.BF = GF(self.p0) # base field
        
        # Constructing RM generator matrix
        self.G_RM = matrix(self.BF, self.k_RM, self.n_RM, lambda i,j : 0)
        
        self.G_RM = self.get_G(self.r_RM, self.m_RM)
        
    
    def get_G(self, r, m):
        n = 2**m
        k = sum(binomial(m,i) for i in range(r + 1))
        d = 2**(m - r)

        #if (n == k):
            # trivial code

        if (n == d):
            # repetition code
            return matrix(self.BF, k, n, lambda i,j : 1)

        elif (d == 2 and n == k + 1):
            # parity check code
            PCC = codes.ParityCheckCode(GF(self.p0), k)
            return PCC.generator_matrix()

        else:
            return block_matrix([ [self.get_G(r,m-1), self.get_G(r,m-1)], [0, self.get_G(r-1,m-1)] ])
        
        
    def Encoding(self, m, zeropad = True):
        
        m = self._IntToPol(m)
        
        rem = len(m) % self.k_RS
        
        if rem != 0:
            if zeropad:
                m.extend([self.EF(0)]*(self.k_RS-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c_RS = []
        
        # Encoding each chunk of size k
        for i in range(0, len(m), self.k_RS):
            c_RS.extend(self.EncodeChunkRS(m[i:i+self.k_RS]))
        
        
        c_RS = self._PolToInt(c_RS)
        c_RS = self._IntToByteString(c_RS)
        
        c_RSRM = []
        
        # Encoding each chunk of size k
        for i in range(0, len(c_RS), self.k_RM):
            c_RSRM.extend(self.EncodeChunkRM(c_RS[i:i+self.k_RM]))
        
        #return vector(self.F, c)
        
        return vector(self.BF, c_RSRM)
            
    def EncodeChunkRS(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RS:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.EF, chunk) * self.G_RS
        return c
    
    def EncodeChunkRM(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RM:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.BF, chunk) * self.G_RM

        return c
    
    def Decoding(self, r):
        
        # Check input size
        if len(r) % self.n_RM != 0:
            raise ValueError('Invalid input size')
            
        c_RM1 = []
        
        for i in range(0,len(r),self.n_RM):
            c_RM1.extend(self.CorrectErrorsRM(r[i:i+self.n_RM], self.r_RM, self.m_RM))
            
        
        c_RM2 = []
        
        for i in range(0,len(c_RM1),self.n_RM):
            c_RM2.extend(self.DecodeChunkRM(c_RM1[i:i+self.n_RM]))
            
        c_RM2 = self._ByteStringToInt(c_RM2)
        c_RM2 = self._IntToPol(c_RM2)
        
        c_RSRM = []
        
        for i in range(0,len(c_RM2),self.n_RS):
            c_RSRM.extend(self.BivariateInterpolation(c_RM2[i:i+self.n_RS]))
            
        c_RSRM = self._PolToInt(c_RSRM)
        return c_RSRM
    
    
    def BivariateInterpolation(self, chunk):
        
        if len(chunk) != self.n_RS:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.EF, self.n_RS, self.tau_RS + self.k_RS, lambda i,j : self.alpha[i]**j)
        M2 = matrix(self.EF, self.n_RS, self.tau_RS + 1, lambda i,j : chunk[i] * self.alpha[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R(list(sol[:self.tau_RS+self.k_RS]))
        Q1 = self.R(list(sol[self.tau_RS+self.k_RS:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            #print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.EF(0)]*(self.k_RS-len(out)))

        return out
    
    
    def CorrectErrorsRM(self, word, r, m):
        
        word = vector(self.BF, word)
        
        if (self.n_RM % len(word)) != 0:
            raise ValueError('Invalid chunk size')
        
        if (r == 0):
            return self.repetition_decode(word)
        
        if (m == r + 1):
            return 'parity check'
        
        l = len(word)
        u1 = word[:l//2]
        u2_v = word[l//2:]
        
        v_error = u1 + u2_v

        v = self.CorrectErrorsRM(v_error, r - 1, m - 1)
        u2 = u2_v + v
        
        u1_decoded = self.CorrectErrorsRM(u1, r, m - 1)
        u2_decoded = self.CorrectErrorsRM(u2, r, m - 1)
        u = 0
        
        if (u1_decoded == 'parity check' and u2_decoded == 'parity check'):
            u = self.parity_check_compare(u1, u2)
            return vector(self.BF, list(u) + list(u + v))
        else:
            return vector(self.BF, list(u1_decoded) + list(u2_decoded + v))
        
    
    
    def repetition_decode(self, word):
        
        if word.hamming_weight() >= len(word.list())//2:
            return vector(self.BF, [1 for i in range(len(word.list()))])
        elif word.hamming_weight() < len(word.list())//2:
            return vector(self.BF, [0 for i in range(len(word.list()))])
        else:
            return "decoding failure"
        
    def parity_check_compare(self, word1, word2):
        decoded1 = word1[:-1]
        decoded2 = word2[:-1]
        
        if (((decoded1.hamming_weight() % 2) == 0 and word1[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word1[-1] == 1)):
            return word1
        elif (((decoded2.hamming_weight() % 2) == 0 and word2[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word2[-1] == 1)):
            return word2
        else:
            return "decoding failure"
        
        
    def DecodeChunkRM(self, chunk):
        cols = self.G_RM.pivots()
        G_RM_independent = self.G_RM.matrix_from_columns(cols)
        chunk_independent = [chunk[i] for i in cols]
        
        return vector(self.BF, chunk_independent) * G_RM_independent.inverse()
    
    
    def _IntToPol(self, m):
        # Convert array of integers less than q to elements of field
        
        m_out = []
        
        for i in m:
            if not i < self.q_RS:
                raise ValueError('Invalid symbol')
            m_out.append(self.EF(ZZ(i).digits(self.p0)))
            
        return m_out
    
    def _PolToInt(self, pol_array):

        # Converts array of integers less than q to elements of Field.

        pol_out = []

        for pol in pol_array:
            if not pol in self.EF:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.p0))

        return(pol_out)

    def _IntToByteString(self, int_array):

        if(self.q_RS != 2^4):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of integers less than 256 to binary representation

        if any([(item > 15 or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        return list(''.join([format(item, '04b') for item in int_array]))
    
    def _ByteStringToInt(self, byte_string):

        if(self.q_RS != 2**4):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of 8 bit binary representations to integers

        #if (len(byte_string) % self.k_RM != 0) or any([not (bit == '1' or bit == '0') for bit in byte_string]):
        #    raise ValueError('Invalid byte string')

        m_out = []
        for i in range(0,len(byte_string), self.k_RM):
            current = byte_string[i:i+self.k_RM]
            m_out.append(int("".join(str(x) for x in current), 2))
            
        return m_out
        
    
    def _IntToPol(self, m):
        # Converts array of integers less than q to elements of Field.

        m_out = []

        for s in m:
            if not s < self.q_RS:
                raise ValueError('Invalid symbol')
            m_out.append(self.EF(ZZ(s).digits(self.p0)))

        return(m_out)

In [297]:
C = RSRMCode(15,7,2**4,1,3)

In [298]:
message = [1,2,3,4,5,6,7]
c = C.Encoding(message)
c

(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0)

In [299]:
e = vector(GF(2), [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
r = c + e
r

(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0)

In [301]:
d = C.Decoding(r)
print(d)

[1, 2, 3, 4, 5, 6, 7]
